

```
 SPRINT 3: EXTRAÇÃO DE ATRIBUTOS - VERSÃO COM MODELO OPEN-SOURCE (LLAVA)
```



# --- Etapa 1: Instalação das Bibliotecas Necessárias ---

In [15]:
# Garante que as dependências para o modelo e otimizações estão instaladas.
!pip install transformers torch bitsandbytes accelerate Pillow -q
print("Dependências instaladas com sucesso.")

Dependências instaladas com sucesso.



# --- Etapa 2: Importações e Configuração do Modelo ---

In [16]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
import requests
import io
import json
import pandas as pd
import re
import warnings
from PIL import Image # <--- CORREÇÃO: A importação que faltava foi adicionada aqui.

warnings.filterwarnings("ignore")
print("Bibliotecas importadas com sucesso.")

Bibliotecas importadas com sucesso.


In [17]:
if not torch.cuda.is_available():
    raise SystemError("GPU não encontrada. Por favor, habilite uma GPU no ambiente de execução.")
print(f"GPU detectada: {torch.cuda.get_device_name(0)}")

GPU detectada: Tesla T4


In [18]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model_id = "llava-hf/llava-1.5-7b-hf"

print(f"Carregando o modelo '{model_id}'...")
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_id)
print("Modelo e processador carregados com sucesso.")

Carregando o modelo 'llava-hf/llava-1.5-7b-hf'...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Modelo e processador carregados com sucesso.


# --- Etapa 3: Definição dos Dados de Entrada e Prompt ---

In [19]:
imagens_cartuchos = [
    {"nome": "hp_664_preto_original.jpg", "url": "https://fotos.oceanob2b.com/High/035033.jpg"},
    {"nome": "toner_brother_tn2370_compativel.jpg", "url": "https://img.kalunga.com.br/fotosdeprodutos/217715z_1.jpg"},
    {"nome": "epson_t544_preto_original.jpg", "url": "https://images.tcdn.com.br/img/img_prod/406359/kit_economico_com_2_garrafas_tinta_epson_544_t544120_t544_preto_original_p_ecotank_l3150_l3160_l3250_5143_1_e5f5b3a89d0946e5093780f45bb3e30c.jpg"},
    {"nome": "canon_cl246_colorido_original.jpg", "url": "https://m.media-amazon.com/images/I/71xOF3-NTjL.jpg"},
    {"nome": "hp_954_amarelo_compativel.png", "url": "https://br-media.hptiendaenlinea.com/catalog/product/cache/b3b166914d87ce343d4dc5ec5117b502/L/0/L0S56AB-1_T1680779403.png"}
]


In [20]:
prompt_text = """
Você é um especialista em analisar imagens de suprimentos de impressora. Analise a imagem fornecida e extraia os atributos do cartucho.
Sua resposta deve ser exclusivamente um objeto JSON bem formatado. Não inclua nenhum texto explicativo antes ou depois do JSON.

O JSON deve conter as seguintes chaves:
- "modelo_cartucho": O número ou código do modelo visível no cartucho (ex: 'HP 664', 'TN2370', 'T544'). Se não for legível, retorne "Não identificado".
- "tipo_cartucho": Classifique como "Original" se tiver logos da marca (HP, Brother, Epson, Canon), "Compatível" se for uma marca de terceiros, ou "Remanufaturado" se houver sinais de reutilização. Se incerto, retorne "Não identificado".
- "cor": Identifique a cor da tinta (ex: 'Preto', 'Ciano', 'Magenta', 'Amarelo', 'Colorido'). Se for um toner, a cor é 'Preto', a menos que especificado de outra forma. Se não for possível determinar, retorne "Não identificado".
- "selo_original_presente": Retorne o valor booleano `true` (sem aspas) se houver um selo holográfico de segurança ou um lacre da marca original. Caso contrário, retorne o valor booleano `false` (sem aspas).
"""

colunas = ["nome_imagem", "url_imagem", "modelo_cartucho", "tipo_cartucho", "cor", "selo_original_presente"]

# --- Etapa 4: Função para Extração com LLaVA ---

In [21]:
def extrair_atributos_com_llava(image_url, prompt):
    print(f"Processando (LLaVA): {image_url.split('/')[-1]}...")
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(image_url, stream=True, headers=headers, timeout=20)
        response.raise_for_status()
        raw_image = Image.open(io.BytesIO(response.content)).convert('RGB')

        prompt_llava = f"USER: <image>\n{prompt} ASSISTANT:"
        inputs = processor(text=prompt_llava, images=raw_image, return_tensors="pt").to("cuda")

        output = model.generate(**inputs, max_new_tokens=250, do_sample=False)
        response_text = processor.decode(output[0], skip_special_tokens=True)

        json_part = response_text.split("ASSISTANT:")[1].strip()
        return json_part
    except requests.exceptions.RequestException as e:
        print(f"  -> ERRO de rede ao baixar a imagem {image_url.split('/')[-1]}: {e}")
        return None
    except Exception as e:
        print(f"  -> ERRO inesperado ao processar a imagem {image_url.split('/')[-1]}: {e}")
        return None

# --- Etapa 5: Execução do Processo e Armazenamento ---

In [22]:
print("\nIniciando a extração de atributos com o modelo LLaVA (versão 5 - funcional)...")
resultados_extraidos = []

for imagem in imagens_cartuchos:
    resposta_texto = extrair_atributos_com_llava(imagem["url"], prompt_text)
    if resposta_texto is None:
        continue

    try:
        match = re.search(r'\{.*\}', resposta_texto, re.DOTALL)
        if not match:
            raise json.JSONDecodeError("Nenhum objeto JSON encontrado na resposta.", resposta_texto, 0)

        resposta_limpa = match.group(0)
        resposta_limpa = resposta_limpa.replace('\\_', '_')

        dados = json.loads(resposta_limpa)
        resultado = {
            "nome_imagem": imagem["nome"],
            "url_imagem": imagem["url"],
            "modelo_cartucho": dados.get("modelo_cartucho", "Erro de parsing"),
            "tipo_cartucho": dados.get("tipo_cartucho", "Erro de parsing"),
            "cor": dados.get("cor", "Erro de parsing"),
            "selo_original_presente": dados.get("selo_original_presente", "Erro de parsing")
        }
        resultados_extraidos.append(resultado)
    except (json.JSONDecodeError, KeyError) as e:
        print(f"  -> ERRO ao processar o JSON da imagem {imagem['nome']}: {e}")
        print(f"     Resposta recebida (original): {resposta_texto}")

print("\nExtração concluída.")

df_resultados = pd.DataFrame(resultados_extraidos, columns=colunas)
nome_arquivo_csv = 'extracao_atributos_llava_v5.csv'
df_resultados.to_csv(nome_arquivo_csv, index=False, encoding='utf-8')
print(f"Resultados salvos com sucesso no arquivo '{nome_arquivo_csv}'")


Iniciando a extração de atributos com o modelo LLaVA (versão 5 - funcional)...
Processando (LLaVA): 035033.jpg...
Processando (LLaVA): 217715z_1.jpg...
Processando (LLaVA): kit_economico_com_2_garrafas_tinta_epson_544_t544120_t544_preto_original_p_ecotank_l3150_l3160_l3250_5143_1_e5f5b3a89d0946e5093780f45bb3e30c.jpg...
Processando (LLaVA): 71xOF3-NTjL.jpg...
Processando (LLaVA): L0S56AB-1_T1680779403.png...

Extração concluída.
Resultados salvos com sucesso no arquivo 'extracao_atributos_llava_v5.csv'


# --- Etapa 6: Exibição Final ---

In [23]:
print("\n--- Tabela de Resultados Extraídos ---")
print(df_resultados.to_string())


--- Tabela de Resultados Extraídos ---
                           nome_imagem                                                                                                                                                                                       url_imagem   modelo_cartucho tipo_cartucho               cor  selo_original_presente
0            hp_664_preto_original.jpg                                                                                                                                                      https://fotos.oceanob2b.com/High/035033.jpg  Não identificado    Compatível             Preto                   False
1  toner_brother_tn2370_compativel.jpg                                                                                                                                         https://img.kalunga.com.br/fotosdeprodutos/217715z_1.jpg            TN2370    Compatível             Preto                    True
2        epson_t544_preto_original.jpg  ht